# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3be0462880>
├── 'a' --> tensor([[ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083]])
└── 'x' --> <FastTreeValue 0x7f3be0462e50>
    └── 'c' --> tensor([[ 1.6224, -0.0354,  0.2774, -0.0305],
                        [-0.2023, -2.0576,  1.7253,  0.2902],
                        [ 1.3890, -0.4198,  0.6423,  0.4596]])

In [4]:
t.a

tensor([[ 0.3338,  0.3585, -0.1206],
        [-1.1269, -0.0033, -0.6083]])

In [5]:
%timeit t.a

76.4 ns ± 1.29 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3be0462880>
├── 'a' --> tensor([[ 0.5425, -0.3483,  1.0465],
│                   [ 0.4359, -0.0565,  0.6000]])
└── 'x' --> <FastTreeValue 0x7f3be0462e50>
    └── 'c' --> tensor([[ 1.6224, -0.0354,  0.2774, -0.0305],
                        [-0.2023, -2.0576,  1.7253,  0.2902],
                        [ 1.3890, -0.4198,  0.6423,  0.4596]])

In [7]:
%timeit t.a = new_value

79.9 ns ± 0.46 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3338,  0.3585, -0.1206],
               [-1.1269, -0.0033, -0.6083]]),
    x: Batch(
           c: tensor([[ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3338,  0.3585, -0.1206],
        [-1.1269, -0.0033, -0.6083]])

In [11]:
%timeit b.a

71.1 ns ± 1.53 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1268, -0.4107, -0.3960],
               [-1.8969,  1.0229, -0.2205]]),
    x: Batch(
           c: tensor([[ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596]]),
       ),
)

In [13]:
%timeit b.a = new_value

631 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.02 µs ± 14.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.3 µs ± 292 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

299 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

275 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3b330c56a0>
├── 'a' --> tensor([[[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]],
│           
│                   [[ 0.3338,  0.3585, -0.1206],
│                    [-1.1269, -0.0033, -0.6083]]])
└── 'x' --> <FastTreeValue 0x7f3b330c5400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

50.7 µs ± 3.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3bb4b5c280>
├── 'a' --> tensor([[ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083],
│                   [ 0.3338,  0.3585, -0.1206],
│                   [-1.1269, -0.0033, -0.6083]])
└── 'x' --> <FastTreeValue 0x7f3be0492a60>
    └── 'c' --> tensor([[ 1.6224, -0.0354,  0.2774, -0.0305],
                        [-0.2023, -2.0576,  1.7253,  0.2902],
                 

In [23]:
%timeit t_cat(trees)

43.7 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.4 µs ± 5.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.6224, -0.0354,  0.2774, -0.0305],
                       [-0.2023, -2.0576,  1.7253,  0.2902],
                       [ 1.3890, -0.4198,  0.6423,  0.4596]],
              
                      [[ 1.6224, -0.0354,  0.2774, -0.0305],
                       [-0.2023, -2.0576,  1.7253,  0.2902],
                       [ 1.3890, -0.4198,  0.6423,  0.4596]],
              
                      [[ 1.6224, -0.0354,  0.2774, -0.0305],
                       [-0.2023, -2.0576,  1.7253,  0.2902],
                       [ 1.3890, -0.4198,  0.6423,  0.4596]],
              
                      [[ 1.6224, -0.0354,  0.2774, -0.0305],
                       [-0.2023, -2.0576,  1.7253,  0.2902],
                       [ 1.3890, -0.4198,  0.6423,  0.4596]],
              
                      [[ 1.6224, -0.0354,  0.2774, -0.0305],
                       [-0.2023, -2.0576,  1.7253,  0.2902],
                       [ 1.3890, -0.4198,  0.6423,  0.4596]],

In [26]:
%timeit Batch.stack(batches)

107 µs ± 3.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596],
                      [ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596],
                      [ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596],
                      [ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596],
                      [ 1.6224, -0.0354,  0.2774, -0.0305],
                      [-0.2023, -2.0576,  1.7253,  0.2902],
                      [ 1.3890, -0.4198,  0.6423,  0.4596],
                      [ 1.6224, -0.0354,  0.2774, -0.0305],
                   

In [28]:
%timeit Batch.cat(batches)

188 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

593 µs ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
